In [36]:
import pandas as pd
import numpy as np
import re
from collections import Counter
import spacy
import nltk
from nltk.corpus import stopwords

In [ ]:
print("A carregar o dataset original")
try:
    df = pd.read_csv("Hotel_Reviews.csv")
    print(f"Dataset Original carregado: {df.shape[0]} linhas, {df.shape[1]} colunas")
except FileNotFoundError:
    print("ERRO: O ficheiro 'Hotel_Reviews.csv' não foi encontrado. Verifica a pasta.")

A carregar o dataset original
Dataset Original carregado: 515738 linhas, 17 colunas


In [ ]:
print("A limpar dados ruidosos...")

# O Booking.com preenche reviews vazias com "No Negative" ou "No Positive".
# Isto confunde a IA (ela acha que a palavra "Negative" está no texto).
# Substituímos por string vazia.
df["Negative_Review"] = df["Negative_Review"].replace("No Negative", "")
df["Positive_Review"] = df["Positive_Review"].replace("No Positive", "")

# Substituir valores vazios reais (NaN) por string vazia
df["Negative_Review"] = df["Negative_Review"].fillna("")
df["Positive_Review"] = df["Positive_Review"].fillna("")

print("Limpeza de textos padrão concluída.")

A limpar dados ruidosos
Limpeza de textos padrão concluída.


In [ ]:
# Conta reviews positivas e negativas por hotel
positivas = df[df['Positive_Review'].str.strip() != ''].groupby('Hotel_Name').size()
negativas = df[df['Negative_Review'].str.strip() != ''].groupby('Hotel_Name').size()

# Seleciona hotéis com pelo menos 15 de cada
hoteis_validos = positivas[(positivas >= 15) & (negativas.get(positivas.index, 0) >= 15)].index

# Filtra o DataFrame
df = df[df['Hotel_Name'].isin(hoteis_validos)]

df = df[(df['Review_Total_Positive_Word_Counts'] > 10) & (df['Review_Total_Negative_Word_Counts'] > 10)]

df['word_count'] = df['Review_Total_Positive_Word_Counts'] + df['Review_Total_Negative_Word_Counts']

# Ordena por riqueza de informação para garantir que as 20 escolhidas são detalhadas
df_sorted = df.sort_values(['Hotel_Name', 'word_count'], ascending=[True, False])

def sample_hotel_reviews(group):
    # Se o hotel tem 40 ou menos reviews, devolvemos todas
    if len(group) <= 40:
        return group
    
    # Se tem mais de 40, pegamos nas 20 com melhor nota e 20 com pior nota
    top_20 = group.nlargest(20, 'Reviewer_Score')
    bottom_20 = group.nsmallest(20, 'Reviewer_Score')
    
    # Concatenar e remover duplicados (caso o top 20 e bottom 20 se sobreponham)
    return pd.concat([top_20, bottom_20]).drop_duplicates()

# 3. Aplicar a amostragem por grupo de hotel
df_final = df_sorted.groupby('Hotel_Name', group_keys=False).apply(sample_hotel_reviews)

# 4. Baralhar os resultados finais para o modelo não ver tudo por ordem alfabética
df_final = df_final.sample(frac=1, random_state=42).reset_index(drop=True)

df_final = df_final.drop(columns=['word_count'])

print(f"Amostragem concluída!")
print(f"Número de hotéis únicos: {df_final['Hotel_Name'].nunique()}")
print(f"Total de reviews final: {len(df_final)}")

Amostragem concluída!
Número de hotéis únicos: 1460
Total de reviews final: 46537


C:\Users\diogo\AppData\Local\Temp\ipykernel_18240\387911594.py:31: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_final = df_sorted.groupby('Hotel_Name', group_keys=False).apply(sample_hotel_reviews)


In [ ]:
def limpar_tags(tag_str):
    if pd.isna(tag_str) or tag_str == "":
        return ""
    
    clean_str = tag_str.replace("[", "").replace("]", "").replace("'", "")
    
    lista_tags = clean_str.split(",")
    
    tags_limpas = [t.strip() for t in lista_tags if t.strip()]
    
    # Devolver como uma string limpa separada por vírgulas ou como lista
    return ", ".join(tags_limpas)

df_final['Tags_Clean'] = df_final['Tags'].apply(limpar_tags)

In [ ]:
def processar_texto(row):
    """
    Combina as reviews positiva e negativa e adiciona contexto (Nome do Hotel e Endereço).
    Não removemos stopwords para manter a semântica para modelos Transformer.
    """
    # Tratamento do conteúdo positivo
    pos = str(row['Positive_Review']).strip()
    texto_positivo = f"O QUE OS CLIENTES ADORAM: {pos}" if pos else "Sem comentários positivos destacados."
    
    # Tratamento do conteúdo negativo
    neg = str(row['Negative_Review']).strip()
    texto_negativo = f"PONTOS A MELHORAR: {neg}" if neg else "Sem queixas relevantes registadas."
    
    # ESTRATÉGIA RAG: Injetar Metadados no Texto 
    # Ajuda a encontrar "Hotel em Londres" porque a palavra "London" passa a fazer parte do texto vetorial.
    texto_final = (
        f"Hotel Name: {row['Hotel_Name']}. "
        f"Location: {row['Hotel_Address']}. "
        f"{texto_positivo} "
        f"{texto_negativo}"
    )
    
    texto_final = " ".join(texto_final.split())
    
    return texto_final

print("Função de processamento definida.")

Função de processamento definida.


In [ ]:
print("A criar a coluna de Texto Enriquecido...")

df_final['review'] = df_final.apply(processar_texto, axis=1)

print(f"Processamento concluído. Linhas válidas restantes: {len(df_final)}")

A criar a coluna de Texto Enriquecido
Processamento concluído. Linhas válidas restantes: 46537


In [ ]:
nlp = spacy.load("en_core_web_sm")
nltk.download('stopwords', quiet=True)
stop_words_nltk = set(stopwords.words('english'))
extra_stops = {
    'hotel', 'room', 'staff', 'stay', 'location', 'would', 'could', 'also', 'get', 'us',
    'loved', 'liked', 'amazing', 'good', 'nice', 'excellent', 'great', 'really', 'bit',
    'everything', 'nothing', 'breakfast', 'egg', 'eggs', 'even', 'next', 'one', 'back', 
    'front', 'desk', 'facilities', 'nearest'
}
ALL_STOPWORDS = stop_words_nltk.union(extra_stops)

# Lista de exclusão definitiva para o NER (Palavras que NUNCA devem ser entidades)
STOP_NER = {
    'breakfast', 'room', 'rooms', 'bed', 'beds', 'shower', 'bathroom', 
    'staff', 'hotel', 'stay', 'everything', 'nothing', 'thing', 'bit',
    'nice', 'great', 'small', 'tiny', 'excellent', 'door', 'window',
    'front', 'desk','facilities', 'nearest', 'egg', 'eggs', 'loved', 
    'really', 'helpful', 'liked'
}

def extrair_conhecimento_hibrido(row):
    texto_pos = str(row['Positive_Review'])
    texto_neg = str(row['Negative_Review'])
    texto_total = (texto_pos + " " + texto_neg).strip()
    
    if len(texto_total) < 10:
        return "", "", ""

    # Keywords
    # Extraímos as palavras mais frequentes que não são stopwords
    palavras = re.findall(r'\b[a-zA-Z]{3,}\b', texto_total.lower())
    palavras_uteis = [p for p in palavras if p not in ALL_STOPWORDS]
    keywords = ", ".join([w for w, f in Counter(palavras_uteis).most_common(8)])

    # NER & POIs
    doc = nlp(texto_total[:1500])

    entidades_geral = []
    pois = []

    for ent in doc.ents:
        ent_text = ent.text.strip()
        ent_lower = ent_text.lower()
        
        # Ignora se a base for Adjetivo/Verbo ou se estiver na lista STOP_NER
        if ent.root.pos_ in ['ADJ', 'VERB'] or ent_lower in STOP_NER:
            continue
        
        # Ignora se for o próprio nome do hotel (evita auto-referência)
        if ent_lower in str(row['Hotel_Name']).lower():
            continue

        # Coluna POI: Apenas locais físicos e natureza (FAC e LOC)
        if ent.label_ in ['FAC', 'LOC']:
            pois.append(ent_text)
            # Se é um POI, tem de estar obrigatoriamente nas entidades_ner
            entidades_geral.append(f"{ent_text} ({ent.label_})")

        # Coluna Entidades: Adiciona também as Organizações e Cidades
        elif ent.label_ in ['ORG', 'GPE']:
            entidades_geral.append(f"{ent_text} ({ent.label_})")

    return (
        keywords,
        ", ".join(list(dict.fromkeys(entidades_geral))[:5]),
        ", ".join(list(dict.fromkeys(pois))[:4])
    )

# Função para verificar a cidade no endereço
def identificar_cidade(address):
    for city in ["London", "Paris", "Amsterdam", "Barcelona", "Milan", "Vienna"]:
        if city in address or (city == "London" and "United Kingdom" in address):
            return city
    return "Other"

print("A processar Keywords, NER, POIs e Cidades...")

df_final[['keywords', 'entidades_ner', 'POI']] = df_final.apply(
    lambda row: pd.Series(extrair_conhecimento_hibrido(row)), axis=1
)

df_final['City'] = df_final['Hotel_Address'].apply(identificar_cidade)

print("Concluído!")

A processar Inteligência Artificial (Keywords, NER, POIs e Cidades)...
Concluído!


In [15]:
colunas_uteis = [
    'Hotel_Name', 
    'Hotel_Address', 
    'City',
    'Average_Score',
    'Total_Number_of_Reviews',
    'Reviewer_Nationality',
    'review',
    'Review_Date',
    'Positive_Review', 
    'Negative_Review',
    'Reviewer_Score',
    'Tags_Clean',
    'keywords',
    'entidades_ner',
    'POI',
    'lat', 
    'lng'
]

df_final = df_final[colunas_uteis]

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

print("A gerar embeddings...")
embeddings = model.encode(df_final['review'].tolist(), show_progress_bar=True)

df_final['embeddings'] = list(embeddings)

df_final.to_pickle("Hotel_Reviews_processed.pkl")
print("Sucesso! O ficheiro .pkl está pronto para ser usado na app.py")

A gerar embeddings...


Batches:   0%|          | 0/1455 [00:00<?, ?it/s]

C:\Users\diogo\AppData\Local\Temp\ipykernel_18240\3204215337.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['embeddings'] = list(embeddings)


Sucesso! Agora usa o ficheiro .pkl na tua app.py


In [ ]:
print(f"A guardar ficheiro final...")
df_final.to_csv("Hotel_Reviews_processed.csv", index=False)
print("Csv criado!")

A guardar ficheiro final...
Csv criado
